In [1]:
import cv2
import torch
import numpy as np
import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights
from scipy.spatial.distance import cosine
from ultralytics import YOLO
import supervision as sv 


In [16]:

# ================= CONFIGURATION =================
# Replace these with your actual file paths
VIDEO_PATH = "test_video/video4.mp4"
OUTPUT_PATH = "detection_results_video/detection_result41.mp4"

# Define your Line coordinates (Start X, Start Y, End X, End Y)
# You will likely need to adjust these based on your camera angle.
# Example: A horizontal line across the middle of a 1920x1080 frame
# LINE_START = (0, 500)
# LINE_END = (1920, 500)

# Visual settings
LINE_COLOR = (0, 0, 255) # Red
TEXT_COLOR = (255, 255, 255)
# =================================================

def run_counter():
    # Load the YOLOv8 model
    model = YOLO('head_detection_best.pt')  # or your custom model path

    video = cv2.VideoCapture(VIDEO_PATH)
    if not video.isOpened():
        print(f"Error: Could not open video file {VIDEO_PATH}")
        exit()

    # Get video properties
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))

    
    LINE_START = (int(frame_width/2), 0)
    LINE_END = (int(frame_width/2), frame_height)


    # Output writer (Using fps/2 because we skip every 2nd frame)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps/3, (frame_width, frame_height))

    # Store the history of track IDs: {id: [previous_center, current_center]}
    track_history = {}
    
    # Counters
    in_count = 0
    out_count = 0
    
    # Set of IDs that have already been counted to prevent double counting
    counted_ids = set()

    print("Processing video...")
    
    frame_index = 0

    while video.isOpened():
        success, frame = video.read()
        
        if not success:
            break

        # Skip every other frame to speed up processing (logic from your snippet)
        frame_index += 1
        if frame_index % 2 == 0:
            continue
        if frame_index % 3 == 0:
            continue

        # 1. Run YOLOv8 tracking
        results = model.track(frame, persist=True, conf=0.65, verbose=False)

        # 2. Extract Data
        # results[0].boxes.id is None if no objects are tracked
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xywh.cpu() # x, y, width, height
            confidence = results[0].boxes.conf.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()


            # Process each tracked object
            for box, track_id, conf in zip(boxes, track_ids, confidence):
                x, y, w, h = box

                center = (int(x), int(y))

                # Update history for this ID
                if track_id not in track_history:
                    track_history[track_id] = []
                
                track_history[track_id].append(center)
                
                # We need at least 2 points (prev and curr) to check crossing
                if len(track_history[track_id]) > 4:
                     # Keep only last 4 points to save memory
                    track_history[track_id] = track_history[track_id][-4:]
                    
                    prev_point = track_history[track_id][0]
                    curr_point = track_history[track_id][3]

                    # 3. Check for Line Crossing
                    if track_id not in counted_ids:
                        # Determine if line was crossed using cross product approach
                        # This determines which "side" of the line a point is on
                        def get_orientation(p1, p2, p3):
                            # Cross product of vector (p2-p1) and (p3-p1)
                            val = (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0])
                            return val

                        # Line points
                        l1 = LINE_START
                        l2 = LINE_END

                        # Check relative orientation of previous and current points vs the line
                        # If signs differ, they are on opposite sides
                        o1 = get_orientation(l1, l2, prev_point)
                        o2 = get_orientation(l1, l2, curr_point)

                        if o1 * o2 < 0: 
                            # Crossing detected! Now ensure it's within the line segment bounds
                            # (Simple bounding box check)
                            line_min_x = min(l1[0], l2[0])
                            line_max_x = max(l1[0], l2[0])
                            line_min_y = min(l1[1], l2[1])
                            line_max_y = max(l1[1], l2[1])

                            if (line_min_x <= center[0] <= line_max_x) or (line_min_y <= center[1] <= line_max_y):
                                # Determine Direction
                                # Positive cross product usually means "Counter Clockwise" or "Right/Below"
                                # You might need to swap these depending on your line direction
                                if o1 > 0:
                                    in_count += 1
                                else:
                                    out_count += 1
                                
                                # Add to counted set so we don't count them again instantly
                                counted_ids.add(track_id)
                                
                                # Visual flair: Draw a circle when counted
                                cv2.circle(frame, center, 20, (0, 255, 0), -1)
                                
                xA = int(x-w/2)
                yA = int(y-h/2)
                xB = int(x+w/2)
                yB = int(y+w/2)
                # xA, yA, xB, yB = map(int, box)
                conf_label = f"Person:{conf:.2f}"
                cv2.rectangle(frame, (xA, yA),(xB, yB),(0, 255, 0), 2)
                cv2.putText(frame, conf_label, (xA, yA-4), 0, 0.6, (0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                cv2.circle(frame, center, 5, (255, 0, 255), -1)
        # 4. Draw Layout
        # Draw the virtual line
        cv2.line(frame, LINE_START, LINE_END, LINE_COLOR, 2)
        
        # Display Stats
        current_visitors = max(0, in_count - out_count)
        
        info_text = [
            f"In: {in_count}",
            f"Out: {out_count}",
            f"Current Visitors: {current_visitors}"
        ]

        # Draw background for text for readability
        cv2.rectangle(frame, (0, 0), (320, 120), (0,0,0), -1)
        
        for idx, text in enumerate(info_text):
            cv2.putText(frame, text, (10, 35 + (idx * 35)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, TEXT_COLOR, 2)

        out.write(frame)

        # Optional: Display live (press q to quit)
        cv2.imshow("Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
           break

    # Cleanup
    video.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Done! Video saved to {OUTPUT_PATH}")
    print(f"Total In: {in_count}, Total Out: {out_count}")

if __name__ == "__main__":
    run_counter()

Processing video...
Done! Video saved to detection_results_video/detection_result41.mp4
Total In: 2, Total Out: 1
